In [1]:
import numpy as np
import pandas as pd 
import json

import MeCab

In [2]:
m = MeCab.Tagger(' -d /usr/lib64/mecab/dic/mecab-ipadic-neologd')

In [3]:
with open("../data/coco_reviews.json", "r") as f:
    json_dict = json.load(f)

In [4]:
def wakachi(text):
    splitted = ' '.join([
        x.split('\t')[0] for x in m.parse(text).splitlines()[:-1] 
            if x.split('\t')[1].split(',')[0] not in ['助詞', '助動詞', '接続詞', '動詞', '記号']
    ])
    return splitted

In [5]:
def set_stopwords():
    with open ("nlp/stopword_japanese.txt", "r") as f:
        stopwords = [line.strip() for line in f]
        stopwords = [ss for ss in stopwords if not ss==u'']

    return stopwords
# set_stopwords()
stopwords = set_stopwords()

In [6]:
def join_list_str(list):
    return ' '.join(list)

def exclude_stopword(text):
    changed_text = [token for token in text.lower().split(" ") if token != "" if token not in stopwords]
    changed_text = join_list_str(changed_text)
    return changed_text

In [66]:
reviews = []
for i in range(1, len(json_dict)+1):
    # print(json_dict[str(i)]["title"])
    # print(json_dict[str(i)]["reviews"])
    reviews_info = json_dict[str(i)]["reviews"]
    all_review_str = join_list_str([exclude_stopword(wakachi(review_info["review"])) for review_info in reviews_info])
    reviews += [all_review_str]
    # print(i)

In [12]:
from sklearn import feature_extraction

In [67]:
vectorizer = feature_extraction.text.TfidfVectorizer(
    decode_error="ignore",
    strip_accents="unicode",
    lowercase=True,
    analyzer="word",
    max_df=0.5,
    min_df=0.1
)

In [68]:
X = vectorizer.fit_transform(reviews).toarray()
index = X.argsort(axis=1)[:, ::-1]
feature_names = np.array(vectorizer.get_feature_names())
feature_words = feature_names[index]

In [69]:
for record in feature_words:
    print(record)

['最後' 'もう' 'メイン' '時代' '演技' '若い' '良かっ' 'やはり' '感動' '大きい' '世代' 'さらに' 'こんな'
 'ストーリー' '相変わらす' '昭和' 'なんと' 'まあ' '前半' '好き' '今日' '名作' '素敵' '全て' '存在' 'キャラ'
 '場面' '当時' '残念' '俳優' '気持ち' '本当に' '役者' 'よく' 'ラスト' '物語' 'その' 'あんまり' 'とんとん'
 '重要' 'これたけ' '微妙' '劇中' '細かい' 'ラストシーン' 'クライマックス' 'かわいい' '一緒' '以来' '価値'
 'そんなに' 'ほとんと' 'っふり' '理由' '中盤' '深く' '是非' 'イイ' 'ホント' '結末' '序盤' '有名' '高い'
 'やっと' '嫌い' '大好き' '退屈' '記憶' '美し' '大人' '凄く' '幸せ' 'とこ' 'いつも' 'テンホ' '役者さん'
 '上手く' '何度' '良さ' '観賞' '子供' '今年' '終盤' '久々' 'なんか' 'すき' '冒頭' '久しふり' 'リアル'
 'エヒソート' '主役' '舞台' 'やっは' '感情' 'しっかり' '魅力' 'セリフ' '後半' '初めて' 'ちゃんと' '本作'
 '美しい' 'すこく' '邦画' 'さすか' '主人公' '台詞' '面白かっ' '特に' 'そんな' '雰囲気' '内容' '日本'
 'やっはり' '最高' '面白い' '一言' '一番' 'ミステリー' 'レヘル' 'メッセーシ' 'ファン' 'モノ' 'ヒット'
 'リアリティ' '一人' '一本' '人々' '丁寧' '上手' '以外' '会話' '体験' '何より' '余韻' '作り' '使い方'
 '俳優さん' '個人的' '傑作' '全く' '全体' '全体的' '全員' '全然' '公開' '共感' '仕方' '仕事' '今作' '予想'
 '上手い' '上映' '不思議' '世界' '世界観' '中心' '主演' '事件' '人間' '事実' '二人' 'ハランス' '人たち'
 '人物' '人生' '人達' 'ハート' 'イマイチ' 'トラマ' 'テーマ' 'こういう' 'こんなに' 'しっくり' 'しない' 'す